In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import datetime 
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import sqlalchemy

init_notebook_mode(connected=True)

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

WINDOW = 22

In [6]:
get_data_frame()

,date,time,open,close,low,high,volume,name
0,2015-08-01,06:00:00,284.000000,279.845023,275.724508,284.000000,3.013764e+02,USDT_BTC
1,2015-08-02,06:00:00,280.000000,286.438075,280.000000,286.438075,1.065639e+02,USDT_BTC
2,2015-08-03,06:00:00,285.438300,288.965644,284.286275,288.965644,7.570679e+01,USDT_BTC
3,2015-08-04,06:00:00,281.104486,288.000000,281.104486,288.000000,1.194722e+01,USDT_BTC
4,2015-08-05,06:00:00,288.200000,287.569800,279.793314,288.965425,9.297527e+00,USDT_BTC
5,2015-08-06,06:00:00,279.793314,283.738175,275.637475,283.738175,1.670897e+02,USDT_BTC
6,2015-08-07,06:00:00,277.228491,275.922500,275.922500,277.228491,6.218237e+01,USDT_BTC
7,2015-08-08,06:00:00,280.999910,263.000000,262.647627,282.235975,1.032695e+03,USDT_BTC
8,2015-08-09,06:00:00,263.000000,268.863437,262.101000,269.758110,4.100076e+02,USDT_BTC
9,2015-08-10,06:00:00,266.219324,266.219324,262.100000,266.219324,9.522162e+00,USDT_BTC


In [4]:
"""
Парсинг данных

"""

def connect(user, password, db, host: str, port: int, echo=False):
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    eng = sqlalchemy.create_engine(url, client_encoding='utf8', echo=echo)
    meta = sqlalchemy.MetaData(bind=eng)
    return eng, meta

def get_data_frame(pair: str = 'USDT_BTC', exchange: str = 'poloniex') -> pd.DataFrame:

    engine, meta = connect(user='postgres', password='password', db='btccandles', host='94.230.125.199', port=16432)
    df = pd.read_sql_query(
        "SELECT date, time, open, close, low, high, volume, pair.\"name\""
        "FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN ("
        "SELECT pair.id FROM pair, exchange WHERE ("
        "SELECT pair.id FROM pair WHERE pair.name = '" + pair +
        "') = pair.alias_id AND pair.exchange_id = (SELECT exchange.id FROM exchange WHERE exchange.\"name\" = '" + exchange + "')) ORDER BY candlestick.date;",
        con=engine)
    return df


"""
Загрузка данных
"""
def load_data(X, seq_len, train_size=1):
    
    amount_of_features = X.shape[1] 
    X_mat = X.as_matrix() 
    
    sequence_length = seq_len + 1 
    data = []
    
    for index in range(len(X_mat) + 1 - sequence_length):
        data.append(X_mat[index: index + sequence_length])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))

    return x_train, y_train

"""
Собираем LSTM
"""

def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

def predictNumDay(num, pathModel, data, scaler):
    m = load_model(pathModel)
    
    prediction = []
    lastbatch = np.array(data[-WINDOW:])
    for i in np.arange(num):    
        res = m.predict([lastbatch.reshape(1,22, 5)])
        prediction.append(scaler.inverse_transform(res))
    
    return np.array(prediction).reshape(num)


In [6]:
"""
typeBlockchain:
USDT_BTC
USDT_LTC
USDT_ETH
USDT_ETC
USDT_XRP
"""

def nextDayPrediction(typeBlockchain, stock, N = 1):    
    
    df = get_data_frame(typeBlockchain, stock)

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    all_df = df.copy()

    x = all_df[['open', 'low', 'high', 'volume']].copy()
    y = all_df['close'].copy()

    x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

    y = y_scaler.fit_transform(y.values.reshape(-1, 1))
    x['close'] = y
    
    X_train, y_train = load_data(x, WINDOW)
    
    #print (X_train.shape, y_train.shape)
    
    model = build_model(input_shape=(WINDOW, 5))
    
    print('START FIT MODEL...')
    
    start = time.time()
    model.fit(X_train, y_train, batch_size=32, epochs=500,
              verbose=1)
    end = time.time()

    print ('Learning time: ', end-start)
    
    today = time.strftime("_%d_%m_%Y")
    
    pathModel = "../models/model_5f_" + typeBlockchain + today +".h5"
    save_model(model, pathModel)
    
    #model = load_model(pathModel)
    
    trainPredict = model.predict(X_train)
    trainPredict = y_scaler.inverse_transform(trainPredict)
    trainY = y_scaler.inverse_transform([y_train])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))
    
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    """
    
    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.legend(loc='best')
    plt.show()
    
    """
    
    lastDate =str(df.date[df.last_valid_index()]).split('-')
    currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])+1)
    predictionDate = pd.date_range(currentData,periods=N)
    predictNday =  (predictNumDay(N, pathModel, x, y_scaler))

    prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)
    
    """Actual = pd.DataFrame(prices, columns=["close"], index=df.date).close
    Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
    pred = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close

    ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
    TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

    iplot([ActualValues,TrainingValues])
    
    our_Predict = go.Scatter( x = prediction.index, y = prediction.predictionPrice, name = 'NextDayValues')

    iplot([our_Predict, ActualValues])"""
    
    return prediction

In [7]:
USDT_BTC = nextDayPrediction('USDT_BTC', 'poloniex' ,N = 1)

START FIT MODEL...
Epoch 1/500
703/703 [==============================] - 1s - loss: 0.0364 - acc: 0.0014        
Epoch 2/500
703/703 [==============================] - 0s - loss: 0.0033 - acc: 0.0014      
Epoch 3/500
703/703 [==============================] - 0s - loss: 0.0019 - acc: 0.0014     
Epoch 4/500
703/703 [==============================] - 0s - loss: 0.0018 - acc: 0.0014     
Epoch 5/500
703/703 [==============================] - 0s - loss: 0.0017 - acc: 0.0014     
Epoch 6/500
703/703 [==============================] - 0s - loss: 0.0013 - acc: 0.0014     
Epoch 7/500
703/703 [==============================] - 0s - loss: 0.0012 - acc: 0.0014      
Epoch 8/500
703/703 [==============================] - 0s - loss: 0.0014 - acc: 0.0014     
Epoch 9/500
703/703 [==============================] - 0s - loss: 0.0016 - acc: 0.0014     
Epoch 10/500
703/703 [==============================] - 0s - loss: 0.0013 - acc: 0.0014     
Epoch 11/500
703/703 [==============================] -

703/703 [==============================] - 0s - loss: 4.9982e-04 - acc: 0.0014     
Epoch 169/500
703/703 [==============================] - 0s - loss: 5.2254e-04 - acc: 0.0014     
Epoch 170/500
703/703 [==============================] - 0s - loss: 4.6271e-04 - acc: 0.0014     
Epoch 171/500
703/703 [==============================] - 0s - loss: 4.5370e-04 - acc: 0.0014         
Epoch 172/500
703/703 [==============================] - 0s - loss: 6.7994e-04 - acc: 0.0014     
Epoch 173/500
703/703 [==============================] - 0s - loss: 7.8409e-04 - acc: 0.0014         
Epoch 174/500
703/703 [==============================] - 0s - loss: 5.5893e-04 - acc: 0.0014         
Epoch 175/500
703/703 [==============================] - 0s - loss: 5.2660e-04 - acc: 0.0014       
Epoch 176/500
703/703 [==============================] - 0s - loss: 4.1710e-04 - acc: 0.0014     
Epoch 177/500
703/703 [==============================] - 0s - loss: 6.1499e-04 - acc: 0.0014     
Epoch 178/500
703/70

703/703 [==============================] - 0s - loss: 3.0856e-04 - acc: 0.0014     
Epoch 334/500
703/703 [==============================] - 0s - loss: 3.3137e-04 - acc: 0.0014     
Epoch 335/500
703/703 [==============================] - 0s - loss: 3.2339e-04 - acc: 0.0014     
Epoch 336/500
703/703 [==============================] - 0s - loss: 3.6583e-04 - acc: 0.0014     
Epoch 337/500
703/703 [==============================] - 0s - loss: 3.5142e-04 - acc: 0.0014     
Epoch 338/500
703/703 [==============================] - 0s - loss: 4.0395e-04 - acc: 0.0014        
Epoch 339/500
703/703 [==============================] - 0s - loss: 3.7024e-04 - acc: 0.0014     
Epoch 340/500
703/703 [==============================] - 0s - loss: 3.7348e-04 - acc: 0.0014       
Epoch 341/500
703/703 [==============================] - 0s - loss: 3.0681e-04 - acc: 0.0014     
Epoch 342/500
703/703 [==============================] - 0s - loss: 2.9481e-04 - acc: 0.0014       
Epoch 343/500
703/703 [====

703/703 [==============================] - 0s - loss: 2.8028e-04 - acc: 0.0014       
Epoch 417/500
703/703 [==============================] - 0s - loss: 3.1987e-04 - acc: 0.0014     
Epoch 418/500
703/703 [==============================] - 0s - loss: 2.7900e-04 - acc: 0.0014     
Epoch 419/500
703/703 [==============================] - 0s - loss: 3.3767e-04 - acc: 0.0014     
Epoch 420/500
703/703 [==============================] - 0s - loss: 3.1245e-04 - acc: 0.0014       
Epoch 421/500
703/703 [==============================] - 0s - loss: 3.0998e-04 - acc: 0.0014     
Epoch 422/500
703/703 [==============================] - 0s - loss: 4.0317e-04 - acc: 0.0014     
Epoch 423/500
703/703 [==============================] - 0s - loss: 3.6729e-04 - acc: 0.0014     
Epoch 424/500
703/703 [==============================] - 0s - loss: 3.2206e-04 - acc: 0.0014     
Epoch 425/500
703/703 [==============================] - 0s - loss: 3.5387e-04 - acc: 0.0014     
Epoch 426/500
703/703 [=======

In [8]:
USDT_BTC

,predictionPrice
2017-07-26,2581.344971


In [ ]:
USDT_LTC = nextDayPrediction('USDT_LTC', 'poloniex',N = 1)

2017-07-26 11:53:02,522 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 11:53:02,523 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,524 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 11:53:02,525 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,531 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 11:53:02,532 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,533 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_ETH = nextDayPrediction('USDT_ETH', 'poloniex',N = 1)

Learning time:  408.56155252456665
Train Score: 3.41 RMSE


In [ ]:
USDT_ETC = nextDayPrediction('USDT_ETC', 'poloniex',N = 1)

2017-07-26 12:06:48,766 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:06:48,767 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,768 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:06:48,769 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,770 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,771 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,776 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_XRP = nextDayPrediction('USDT_XRP', 'poloniex',N = 1)

2017-07-26 12:10:19,342 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:10:19,343 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,351 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:10:19,352 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,353 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [86]:
print ('predictionValues on', datetime.date.today())

allCurrency = ['USDT_BTC', 'USDT_LTC', 'USDT_ETH', 'USDT_ETC', 'USDT_XRP']
predictedValues = [USDT_BTC, USDT_LTC, USDT_ETH, USDT_ETC, USDT_XRP]

for index in np.arange(len(allCurrency)):
    print(allCurrency[index], '\n' , predictedValues[index] )


predictionValues on 2017-07-26
USDT_BTC 
             predictionPrice
2017-07-26      2545.130371
USDT_LTC 
             predictionPrice
2017-07-26        42.890644
USDT_ETH 
             predictionPrice
2017-07-26       193.827957
USDT_ETC 
             predictionPrice
2017-07-26        14.239629
USDT_XRP 
             predictionPrice
2017-07-26         0.177823


In [63]:
def checkTrend(typeBlockchain, stock, predicted):
    
    df = get_data_frame(typeBlockchain, stock)
    lastValue = df.close[df.shape[0]-1]
    change = (predicted.values * 100 / lastValue) - 100
    if change > 0:
        mark = 'PositiveTrend'
    else: mark = 'NegativeTrend'
        
    print ('-'*10 + '\nSTOCK:%s, type:%s\n'%(stock, typeBlockchain))
    print ('Change between last (%f) and predicted values(%f) = %f perc.\n%s' \
           % (lastValue, predicted.values, change, mark))

In [84]:
stock = 'poloniex'

for index in np.arange(len(allCurrency)):
    checkTrend(allCurrency[index], stock , predictedValues[index] )

----------
STOCK:poloniex, type:USDT_BTC

Change between last (2565.999999) and predicted values(2545.130371) = -0.813316 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_LTC

Change between last (42.110000) and predicted values(42.890644) = 1.853821 perc.
PositiveTrend
----------
STOCK:poloniex, type:USDT_ETH

Change between last (203.000000) and predicted values(193.827957) = -4.518250 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_ETC

Change between last (14.620000) and predicted values(14.239629) = -2.601715 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_XRP

Change between last (0.174540) and predicted values(0.177823) = 1.881226 perc.
PositiveTrend
